In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from preprocessor import Preprocessor
from evaluator import ModelEvaluator
import pandas as pd

In [2]:
"""
Применяется таргет энкодинг как способ обработки категориальных фичей, для nan использовали замещение средним
"""

'\nПрименяется таргет энкодинг как способ обработки категориальных фичей, для nan использовали замещение средним\n'

In [3]:
df_train = pd.read_csv("C:/Users/Роман/Chepenkov_dz2/application_train.csv")
df_test = pd.read_csv("C:/Users/Роман/Chepenkov_dz2/application_test.csv")

In [9]:
def hypothesis_2(df):
    df_encoded = df.copy()
    cat_cols = df.select_dtypes(include=['object', 'string']).columns
    
    train_target = df_train['TARGET']  # global
    
    for col in cat_cols:
        # Используем глобальный train_target для группировки
        encoding_map = train_target.groupby(df[col]).mean().to_dict()
        df_encoded[col] = df[col].map(encoding_map)
    
    return df_encoded

In [18]:
preprocessor = Preprocessor(custom_steps=[hypothesis_2])

X_train_processed = preprocessor.transform(df_train.drop(columns=['TARGET', 'SK_ID_CURR']))
num_cols = X_train_processed.select_dtypes(include='number').columns
X_train_processed = X_train_processed[num_cols]

X_test_processed = preprocessor.transform(df_test.drop(columns=['SK_ID_CURR']))
X_test_processed = X_test_processed[num_cols]

imputer = SimpleImputer(strategy='mean')
X_train_processed = imputer.fit_transform(X_train_processed)
X_test_processed = imputer.transform(X_test_processed)

In [19]:
evaluator = ModelEvaluator(LogisticRegression(max_iter=100))
roc_auc = evaluator.evaluate(X_train_processed, df_train['TARGET'])
print(f"ROC-AUC train: {roc_auc:.5f}")

evaluator.save_test_predictions(
    X_test=X_test_processed,
    ids=df_test['SK_ID_CURR'],
    filename='test_predictions_hyp_2.csv'
)
#ROC-AUC train: 0.62235

#ROC-AUC test: 0.61513
#ROC-AUC test baseline: 0.61204
#ROC-AUC test surplus: 0,00309

C:\Users\Роман\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ROC-AUC train: 0.62235
